In [1]:
import numpy as np
import sys, os, shutil
import pandas as pd
import collections as cl
import itertools as it
import timeit
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import copy as cp
import pyprojroot as pr
import random as rand
import math as m

In [2]:
User=['Emory','WG'][0]
print(User)

Emory


In [3]:
#Change directory
if User=='Emory':
    os.chdir(pr.here("./Data/fake"))
elif User == 'WG':
    os.chdir(pr.here("./Data/real"))
os.getcwd()

'C:\\Users\\yche465\\Desktop\\AIM1\\Codes\\PrEP-Traj-Clustering\\Data\\fake'

In [4]:
#Import original prescription date records
prescrip_dates_orig=pd.read_csv("syndata_13k.csv").astype({'ID':'int'})
#convert the column's data type to date time 
prescrip_dates_orig['PrEP_Start']= pd.to_datetime(prescrip_dates_orig['PrEP_Start'])
prescrip_dates_orig['PrEP_End']= pd.to_datetime(prescrip_dates_orig['PrEP_End'])

In [5]:
#Import sampled patient ID
Sampled_SeroProtect = np.loadtxt('SeroProtect_4k.txt', usecols=range(104), delimiter=",")
Sampled_ID=Sampled_SeroProtect[:,0].astype('int32')

In [6]:
#Extract prescription record for sampled pateints
prescrip_dates_sampled=pd.DataFrame(prescrip_dates_orig.loc[prescrip_dates_orig['ID'].isin(Sampled_ID)])

In [7]:
prescrip_dates_sampled

,ID,PrEP_Start,PrEP_End
14,206,2023-06-26,2023-10-17
15,206,2024-01-04,2024-02-24
16,207,2023-09-06,2023-12-12
19,265,2023-04-05,2023-08-17
27,442,2023-01-14,2023-06-09
...,...,...,...
19777,191726,2023-08-18,2023-11-17
19784,191794,2023-02-18,2023-06-19
19785,191800,2023-09-12,2024-02-13
19787,191819,2023-12-18,2024-03-04


In [8]:
#Create a column for PrEP assessment interval: 
## the time interval bounded by two consecutive PrEP prescription fill dates, 
## or the end date of the two-year tracking period 

startdate=prescrip_dates_sampled.groupby("ID").agg('min').drop(columns=['PrEP_End'])
startdate=startdate.rename(columns={'PrEP_Start':'Initiation'})
prescrip_dates_sampled2=prescrip_dates_sampled.merge(startdate, on="ID",how='left')
prescrip_dates_sampled2['Two_Year_Mark']=prescrip_dates_sampled2['Initiation']+dt.timedelta(days=365*2)

In [9]:
prescrip_dates_sampled2

,ID,PrEP_Start,PrEP_End,Initiation,Two_Year_Mark
0,206,2023-06-26,2023-10-17,2023-06-26,2025-06-25
1,206,2024-01-04,2024-02-24,2023-06-26,2025-06-25
2,207,2023-09-06,2023-12-12,2023-09-06,2025-09-05
3,265,2023-04-05,2023-08-17,2023-04-05,2025-04-04
4,442,2023-01-14,2023-06-09,2023-01-14,2025-01-13
...,...,...,...,...,...
6048,191726,2023-08-18,2023-11-17,2023-08-18,2025-08-17
6049,191794,2023-02-18,2023-06-19,2023-02-18,2025-02-17
6050,191800,2023-09-12,2024-02-13,2023-09-12,2025-09-11
6051,191819,2023-12-18,2024-03-04,2023-12-18,2025-12-17


In [10]:
diff_Pstart_2yrMark=prescrip_dates_sampled2['Two_Year_Mark']-prescrip_dates_sampled2['PrEP_Start']
diff_Pend_2yrMark=prescrip_dates_sampled2['Two_Year_Mark']-prescrip_dates_sampled2['PrEP_Start']

def is_within_2_years(diff):
    return (diff <= dt.timedelta(days=365*2)) and (diff >= dt.timedelta(days=0))



within_bound= diff_Pend_2yrMark.apply(is_within_2_years) & diff_Pstart_2yrMark.apply(is_within_2_years)

In [11]:
prescrip_dates_sampled3=prescrip_dates_sampled2.loc[within_bound,:]
#no rows were out of the 2-year bound
print(prescrip_dates_sampled3.shape==prescrip_dates_sampled2.shape)

#create a column indicating next PrEP start date per row
prescrip_dates_sampled3['Next_PrEP_Start']=prescrip_dates_sampled3.groupby('ID').shift(-1)['PrEP_Start']

#create a column for the end date of PrEP assessment interval
prescrip_dates_sampled3['PrEP_assessment_interval']=np.where(pd.isnull(prescrip_dates_sampled3['Next_PrEP_Start']),
                                                            prescrip_dates_sampled3['Two_Year_Mark'],
                                                            prescrip_dates_sampled3['Next_PrEP_Start'])
#create a column for the end date of PrEP consumption interval
prescrip_dates_sampled3['PrEP_consumption_interval']=np.where(prescrip_dates_sampled3['PrEP_assessment_interval']>prescrip_dates_sampled3['PrEP_End']+dt.timedelta(days=7),
                                                             prescrip_dates_sampled3['PrEP_End']+dt.timedelta(days=7),
                                                             prescrip_dates_sampled3['PrEP_assessment_interval'])

True


In [12]:
prescrip_dates_sampled3

,ID,PrEP_Start,PrEP_End,Initiation,Two_Year_Mark,Next_PrEP_Start,PrEP_assessment_interval,PrEP_consumption_interval
0,206,2023-06-26,2023-10-17,2023-06-26,2025-06-25,2024-01-04,2024-01-04,2023-10-24
1,206,2024-01-04,2024-02-24,2023-06-26,2025-06-25,NaT,2025-06-25,2024-03-02
2,207,2023-09-06,2023-12-12,2023-09-06,2025-09-05,NaT,2025-09-05,2023-12-19
3,265,2023-04-05,2023-08-17,2023-04-05,2025-04-04,NaT,2025-04-04,2023-08-24
4,442,2023-01-14,2023-06-09,2023-01-14,2025-01-13,2023-09-21,2023-09-21,2023-06-16
...,...,...,...,...,...,...,...,...
6048,191726,2023-08-18,2023-11-17,2023-08-18,2025-08-17,NaT,2025-08-17,2023-11-24
6049,191794,2023-02-18,2023-06-19,2023-02-18,2025-02-17,NaT,2025-02-17,2023-06-26
6050,191800,2023-09-12,2024-02-13,2023-09-12,2025-09-11,NaT,2025-09-11,2024-02-20
6051,191819,2023-12-18,2024-03-04,2023-12-18,2025-12-17,NaT,2025-12-17,2024-03-11


In [13]:
#extract ID for those eligible for PrEP nondaily dosing regimen
prescrip_dates_sampled3['ND_eligible']=prescrip_dates_sampled3['PrEP_consumption_interval']<prescrip_dates_sampled3['PrEP_assessment_interval']
DAT=prescrip_dates_sampled3.groupby('ID').agg('max')
ND_Elig_ID=np.array(DAT.index.values.tolist())[DAT['ND_eligible'].tolist()]

In [14]:
#Select 10% of PrEP users to follow nondaily dosing regimen
##Non-daily PrEP users will be randomly selected from the subset of PrEP users who have >=1 PrEP assessment intervals
##in which the interval lengths are greater than the length of the adherent PrEP consumption periods
rand.seed(30)
ID_NDUsers=sorted(rand.sample(sorted(ND_Elig_ID),int(len(Sampled_ID)*0.1)))

In [15]:
#4 doses per week regimen: they would use the non-daily PrEP regimen for the PrEP assessment intervals 
# in which the interval lengths were greater than those of the adherent PrEP consumption periods.

In [16]:
prescrip_dates_sampled3

,ID,PrEP_Start,PrEP_End,Initiation,Two_Year_Mark,Next_PrEP_Start,PrEP_assessment_interval,PrEP_consumption_interval,ND_eligible
0,206,2023-06-26,2023-10-17,2023-06-26,2025-06-25,2024-01-04,2024-01-04,2023-10-24,True
1,206,2024-01-04,2024-02-24,2023-06-26,2025-06-25,NaT,2025-06-25,2024-03-02,True
2,207,2023-09-06,2023-12-12,2023-09-06,2025-09-05,NaT,2025-09-05,2023-12-19,True
3,265,2023-04-05,2023-08-17,2023-04-05,2025-04-04,NaT,2025-04-04,2023-08-24,True
4,442,2023-01-14,2023-06-09,2023-01-14,2025-01-13,2023-09-21,2023-09-21,2023-06-16,True
...,...,...,...,...,...,...,...,...,...
6048,191726,2023-08-18,2023-11-17,2023-08-18,2025-08-17,NaT,2025-08-17,2023-11-24,True
6049,191794,2023-02-18,2023-06-19,2023-02-18,2025-02-17,NaT,2025-02-17,2023-06-26,True
6050,191800,2023-09-12,2024-02-13,2023-09-12,2025-09-11,NaT,2025-09-11,2024-02-20,True
6051,191819,2023-12-18,2024-03-04,2023-12-18,2025-12-17,NaT,2025-12-17,2024-03-11,True


In [17]:
#de-identify the date
a=prescrip_dates_sampled3["PrEP_Start"] - prescrip_dates_sampled3["Initiation"] 
prescrip_dates_sampled3['PrEP_start_days']=a.dt.days
a=prescrip_dates_sampled3["PrEP_End"] - prescrip_dates_sampled3["Initiation"] 
prescrip_dates_sampled3['PrEP_end_days']=a.dt.days
a=prescrip_dates_sampled3['Next_PrEP_Start'] - prescrip_dates_sampled3["Initiation"] 
prescrip_dates_sampled3['Next_PrEP_start_days']=a.dt.days
a=prescrip_dates_sampled3['Two_Year_Mark'] - prescrip_dates_sampled3["Initiation"] 
prescrip_dates_sampled3['Two_Year_Mark_days']=a.dt.days
prescrip_days_sampled=prescrip_dates_sampled3[['ID','PrEP_start_days','PrEP_end_days','Next_PrEP_start_days','Two_Year_Mark_days','ND_eligible']]

#create a column for medication days supply variable
prescrip_days_sampled['Med_days_supply']=prescrip_days_sampled['PrEP_end_days']-prescrip_days_sampled['PrEP_start_days']

#create a column for hypothetical upperbound when medication would run out using doses/week dosing frequency
prescrip_days_sampled['PrEP_use_UB']=round(prescrip_days_sampled['PrEP_start_days']+prescrip_days_sampled['Med_days_supply']*7/4).astype(int)

#create column for the day range within which days of PrEP intake would be sampled
prescrip_days_sampled['PrEP_use_range']=prescrip_days_sampled.apply(lambda x: x['PrEP_use_UB'] if x['Next_PrEP_start_days']> x['PrEP_use_UB'] or m.isnan(x['Next_PrEP_start_days'])
                            else x['Next_PrEP_start_days']-1, axis=1).astype(int)

C:\Users\yche465\AppData\Local\Temp\ipykernel_22940\2755120122.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prescrip_days_sampled['Med_days_supply']=prescrip_days_sampled['PrEP_end_days']-prescrip_days_sampled['PrEP_start_days']
C:\Users\yche465\AppData\Local\Temp\ipykernel_22940\2755120122.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prescrip_days_sampled['PrEP_use_UB']=round(prescrip_days_sampled['PrEP_start_days']+prescrip_days_sampled['Med_days_supply']*7/4).astype(int)
C:\Users\yche465\

In [18]:
prescrip_days_sampled.head(10)

,ID,PrEP_start_days,PrEP_end_days,Next_PrEP_start_days,Two_Year_Mark_days,ND_eligible,Med_days_supply,PrEP_use_UB,PrEP_use_range
0,206,0,113,192.0,730,True,113,198,191
1,206,192,243,NaN,730,True,51,281,281
2,207,0,97,NaN,730,True,97,170,170
3,265,0,134,NaN,730,True,134,234,234
4,442,0,146,250.0,730,True,146,256,249
5,442,250,500,NaN,730,True,250,688,688
6,484,0,90,NaN,730,True,90,158,158
7,500,0,633,NaN,730,True,633,1108,1108
8,540,0,195,374.0,730,True,195,341,341
9,540,374,470,543.0,730,True,96,542,542


In [19]:
#prescription day dataset for the records where days of PrEP intake need to be sampled
prescrip_NDUse=prescrip_days_sampled.loc[prescrip_days_sampled['ID'].isin(ID_NDUsers) & prescrip_days_sampled['ND_eligible']==True,:]
prescrip_NDUse

,ID,PrEP_start_days,PrEP_end_days,Next_PrEP_start_days,Two_Year_Mark_days,ND_eligible,Med_days_supply,PrEP_use_UB,PrEP_use_range
0,206,0,113,192.0,730,True,113,198,191
1,206,192,243,NaN,730,True,51,281,281
14,574,0,73,NaN,730,True,73,128,128
19,745,0,171,260.0,730,True,171,299,259
20,745,260,460,NaN,730,True,200,610,610
...,...,...,...,...,...,...,...,...,...
5962,188818,0,81,197.0,730,True,81,142,142
5980,189512,0,351,NaN,730,True,351,614,614
5986,189730,0,129,NaN,730,True,129,226,226
5989,189971,0,142,NaN,730,True,142,248,248


In [24]:
#Create a PrEP intake matrix for nondaily users (400*730)
medcover_NDUsers=np.zeros((len(ID_NDUsers),365*2))

In [22]:
#Create a list that contains all the randomly selected days for PrEP intake
rand.seed(10)
Days_PrEP_Intake=[]
for i in range(len(ID_NDUsers)):
    #print(ID_NDUsers[i])
    MedUseSupply=prescrip_NDUse.loc[prescrip_NDUse['ID']==ID_NDUsers[i],'Med_days_supply'].tolist()
    PrEP_start_days=prescrip_NDUse.loc[prescrip_NDUse['ID']==ID_NDUsers[i],'PrEP_start_days'].tolist()
    PrEP_use_range=prescrip_NDUse.loc[prescrip_NDUse['ID']==ID_NDUsers[i],'PrEP_use_range'].tolist()
    
    dpi=[]
    for j in range(len(PrEP_start_days)):
        x=rand.sample(range(PrEP_start_days[j], PrEP_use_range[j]+1), MedUseSupply[j])
        x.sort()
        x2=[y for y in x if y <= (len(medcover_NDUsers[0])-1)]
        dpi.append(x2)
    Days_PrEP_Intake.append(dpi)    


In [23]:
#Update the daily PrEP intake matrix using the list we just created
for k in range(len(ID_NDUsers)):
    #print(k)
    for j in range(len(Days_PrEP_Intake[k])):
        dpi=Days_PrEP_Intake[k][j]
        medcover_NDUsers[k][dpi]=1

In [443]:
#Create binary daily intake/coverage matrix for everyone (4000*730)
id_pool=np.unique(prescrip_days_sampled["ID"])
medcover=np.zeros((len(id_pool),365*2))
for k in range(len(id_pool)):
    Rx_start=np.array(prescrip_days_sampled.loc[prescrip_days_sampled["ID"]==id_pool[k],"PrEP_start_days"])
    Rx_end=np.array(prescrip_days_sampled.loc[prescrip_days_sampled["ID"]==id_pool[k],"PrEP_end_days"])
    
    for j in range(len(Rx_start)):
        medcover[k][Rx_start[j]:Rx_end[j]+1]=1

In [456]:
#Update the part of the entire daily intake matrix that belong to non-daily PrEP users 
rownumber_NDUsers=[i for i in range(len(id_pool)) if id_pool[i] in ID_NDUsers]
medcover[rownumber_NDUsers,]=medcover_NDUsers

In [465]:
print(rownumber_NDUsers)

[0, 10, 13, 17, 20, 27, 31, 44, 56, 72, 83, 84, 88, 97, 99, 104, 106, 107, 118, 119, 124, 134, 152, 179, 184, 199, 259, 264, 274, 279, 281, 289, 298, 299, 302, 313, 332, 336, 383, 390, 394, 404, 416, 423, 436, 439, 445, 458, 460, 465, 467, 469, 471, 487, 488, 492, 495, 504, 534, 551, 554, 564, 568, 571, 572, 589, 591, 592, 595, 619, 655, 656, 660, 680, 684, 702, 711, 716, 719, 732, 739, 740, 756, 764, 770, 773, 781, 785, 788, 790, 797, 801, 804, 829, 833, 859, 868, 881, 919, 931, 943, 985, 1002, 1003, 1005, 1027, 1030, 1041, 1048, 1055, 1059, 1062, 1080, 1093, 1099, 1124, 1136, 1143, 1150, 1156, 1171, 1177, 1190, 1195, 1196, 1209, 1223, 1235, 1237, 1243, 1245, 1256, 1275, 1277, 1282, 1287, 1299, 1324, 1326, 1328, 1367, 1371, 1372, 1373, 1381, 1394, 1412, 1419, 1438, 1446, 1447, 1461, 1469, 1525, 1537, 1538, 1539, 1551, 1568, 1570, 1576, 1600, 1606, 1607, 1610, 1615, 1624, 1629, 1637, 1647, 1655, 1656, 1677, 1680, 1689, 1708, 1710, 1723, 1731, 1753, 1767, 1777, 1779, 1805, 1818, 1824, 1

In [498]:
#Test whether the matrix is correctly created
pos=np.where(id_pool==ID_NDUsers[15])[0][0]
print("ID=",ID_NDUsers[15], "is a nondaily user. It's located in Row", pos, "of the daily intake matrix")
print("Check if the daily intake of that row matches the selected daily intake days for ID",ID_NDUsers[15])

medcover[pos,Days_PrEP_Intake[15][0]]==1


ID= 4419 is a nondaily user. It's located in Row 104 of the daily intake matrix
Check if the daily intake of that row matches the selected daily intake days for ID 4419


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [499]:
#Create PMC matrix: % of the 14 days with medication intake/coverage
start=np.array(range(0,103*7,7))
end=start+14
PMC=np.zeros((len(id_pool),103))
for k in range(len(id_pool)):    
    for i in range(len(start)):
        PMC[k][i]=sum(medcover[k][start[i]:end[i]])/14
#Create binary PrEP sero-protection matrix (0: PMC<0.57, 1: otherwise)  
SeroProtect=np.multiply(PMC>=4/7,1)

In [507]:
#Append subject ID column to the SeroProtect matrix
SeroProtect_indexed=np.column_stack((id_pool,SeroProtect))

In [512]:
#Export PMC matrix
f= open("SA_PMC_4k.txt","w+")
for i in range(len(PMC)):
    for j in range(103):
        if j == 102:
            f.write("%s\n" % (PMC[i][j]))
        else:
            f.write("%s," % (PMC[i][j]))
f.close()

In [513]:
#Export sero-protection matrix
f= open("SA_SeroProtect_4k.txt","w+")
for i in range(len(SeroProtect_indexed)):
    for j in range(104):
        if j == 103:
            f.write("%s\n" % (SeroProtect_indexed[i][j]))
        else:
            f.write("%s," % (SeroProtect_indexed[i][j]))
f.close()